# Google's PageRank algorithm

Pagerank, Google's original killer app, is based on eigen-decomposition. Let's see how it works.

*Note: Original notebook from Jonas Kersulis, University of Michigan: https://github.com/kersulis/551-pagerank*
___

## Before we begin

Run the following cell to install and load the packages we will be using.

In [ ]:
from surfer import *
import pandas as pd

%pylab inline

Using the code in `surfer`, you can generate a graph by crawling the internet. See original notebook for example. *For the purpose of this activity, we will load a stored version of the web-site link graph instead, which has been crawled by starting at `http://eecs.umich.edu` and crawling 500 pages.*

In [ ]:
# for reading from file
G = nx.read_gml("data/eecs.gml")

f = open('data/links.txt')
links = f.read().splitlines()
f.close()

In [ ]:
A = nx.adjacency_matrix(G, links).todense()

We will use `spy(A)` to see the connectivity pattern of the adjacency matrix of the web-ste graph.

Note that `A` is an n-by-n matrix with `A[i,j] = 1` if site `i` is linked to node `j`.

In [ ]:
fig = figure(figsize=(6,6))
spy(A, markersize=1, alpha=0.5)

There are a couple intuitive ways to rank the web pages.

1. **Rank by number of incoming links (in degree):** `sum(A,0)` produces a row vector by summing together the rows of the matrix (computing a row sum); each entry of this vector is the number of other sites that links to the corresponding site).
2. **Rank by number of outgoing links (out degree):** `sum(A,1)` produces a column vector; each entry of this vector is the number of sites linked to by the corresponding node.

When we sum $A$ by rows or columns, we don't automatically see the links. The graph package we're using makes it easy to see the links along with in or out degree. Run the following cell to see the ten sites with the most incoming links. You can do the same for out degree by replacing `G.in_degree_iter` with `G.out_degree_iter`.

In [ ]:
# display links sorted by in degree:
din_sort = sorted(G.in_degree_iter(), key=lambda i: i[1], reverse=True)
din_sort[:10] # top 10 sites by in degree

**Which site is linked to by the most other sites? Which site links to the most other sites?**

Do the numbers make sense given the total number of sites (`len(G.nodes()`) and links between them (`len(G.edges()`)?

Which of these rankings seems better? How many sites have no outgoing links, and how should they be ranked?

## Another way to rank pages
The ranking approaches just discussed are easily exploited.

* With in degree ranking, someone might set up a thousand pages that link to one page, thereby inflating its rank.
* With out degree ranking, someone might include a mountain of links on every page they make to inflate its rank.

We know there is a deeper meaning in our internet graph. Some pages are more important than others. Degree ranking misses a key component of website importance: the number of sites linking to a particular site matters less than the _importance_ of those sites.

Our third approach, pagerank, captures this underlying meaning by focusing on probability rather than degree. If one visits site A, what is the probability they will visit site B next? We can infer these probabilities from the adjacency matrix of the graph. Given these probabilities, we can find the page rank of each node by an iterative procedure,
$$
PR^n(p_i)=\alpha\sum_{p_j\in In(p_i)} \left ( PR^{n-1}(p_j)\times \frac{1}{|Out(p_j)|} \right ) + (1-\alpha)tp,
$$
where $n$ is the number of nodes in the graph, $PR^n(p_i)$ and $PR^{n-1}(p_i)$ are the current and last iteration pagerank scores for node/page $p_i$, $In(p_i)$ and $Out(p_i)$ are the in- and out-degrees of the node, respectively, $(1-\alpha)$ is a dampening factor with $0 < \alpha < 1$, and $tp$ is the teleportation probability (i.e., the probability of jumping to a random node instead of following an outgoing link, usually $1/n$). Generally, $\alpha$ is close to 1, e.g., 0.8 or 0.9. Note that $\frac{1}{|Out(p_j)|}$ is in fact the probability of transition from $p_j$ to $p_i$.

If we put all pagerank scores in a vector, we can write the above formula as,
$$
\mathbf{r}^n = \alpha \mathbf{H} \mathbf{r}^{n-1} + (1-\alpha)\mathbf{1}/n,
$$
where $\mathbf{r}^n$ and $\mathbf{r}^{n-1}$ are the pagerank vectors from the current and previous iterations, $\mathbf{H}$ is the transition probablity matrix derived from the graph adjacency matrix, and $\mathbf{1}$ is a vector of all 1's of the appropriate size. We set the initial transition probabilities for all nodes to $1/n$, i.e., $\mathbf{r}^0 = \mathbf{1}/n$.

Given enough iterations, the series converges to the leading eigenvector of the matrix
$$
\alpha H + (1-\alpha)\mathbf{1}/n,
$$
which the original [paper](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) from Google solved via the `power iteration` method, which we will see in a bit. First, we must deal with edge-cases.

## Dangling nodes
Nodes that have an incoming link but no outgoing link are referred to as *dangling nodes* and can cause problems in computing the series (why?). We can "fix" this by forcing every page to be connected to itself, allowing each page to have at least one in-link and out-link. The adjacency matrix produced by `surfer()` will only have a 1 in the `[i,i]` entry if a page explicitly links to itself, so our web adjacency matrix may have dangling nodes. The function belowsets the diagonal of the adjacency matrix to 1.

In [ ]:
def fix_dangling_nodes(A):
    """
    Return input matrix A (square) with all diagonal elements set to 1.
    """
    return tril(A, -1) + triu(A, 1) + eye(size(A, 0))

## Computing the probability matrix
Our goal is to have a matrix where each element $H[i,j]$ is the probability of transitioning from node $i$ to node $j$. Let $G$ be the adjacency matrix with dangling nodes fixed. Then $H$ is generated from $G$ by normalizing the rows to have sum 1.


In [ ]:
def normalize_rows(G):
    """
    Return input matrix A normalized so each row
    represents probability of leaving node
    (each row of returned matrix sums to one)
    """
    out_degree = sum(G, 1)
    H = copy(G)
    for i in range(len(out_degree)):
        if out_degree[i] > 0:
            H[i,:] = G[i,:]/out_degree[i]
    return H

## Eliminating 0 elements

We're almost ready to apply power iteration, but first we need to address the zeros in our probability matrix. The pagerank vector is defined as the leading eigenvector of the matrix

$$
\alpha H + (1-\alpha)\mathbf{1}/n.
$$

To ensure this matrix can be properly de-composed, we replace all 0 probabilities with a very small value.

In [ ]:
def remove_zeros(H, alpha):
    return alpha*H + (1 - alpha)*ones(shape(H))/size(H,0)

## Putting it all together: adjacency to probability

In [ ]:
H = normalize_rows(fix_dangling_nodes(A))
alpha = 0.85
Ha = remove_zeros(H, alpha).T

Note that we took the transpose (by adding `.T`), so now $H[i,j]$ is the probability of transitioning from node $j$ to node $i$. You should be comfortable with manipulations like this -- we're only doing it so we can work with right Eigenvectors rather than left.

## Power iteration
We're going to do what Google first did: use power iteration to compute the leading eigenvector of $H_\alpha$, and use this vector to rank our web pages.

In [ ]:
def power_iteration(Ha, steps):
    """
    Beginning with random starting vector, perform specified
    number of power iteration steps. Return final vector.
    """
    u = rand(size(Ha, 1))
    u = u/norm(u)

    for idx in range(steps):
        u = dot(Ha, u)
        u = u/norm(u)
    return u

In [ ]:
r = power_iteration(Ha, 200)

## Comparing rankings
Run the following cell to create and show a table containing all out in degree, out degree, and pagerank data. Click on a column heading to sort by that column (double-click to sort in descending order).

In [ ]:
din_vec = [G.in_degree()[l] for l in links]
dout_vec = [G.out_degree()[l] for l in links]

df = pd.DataFrame({ 'in_degree' : din_vec,
                    'out_degree' : dout_vec,
                    'pagerank' : r }, index=links)

In [ ]:
# display top 10 sites by each sorting method

df.sort_values(by='in_degree', ascending=False).head(10)
# df.sort_values(by='out_degree', ascending=False).head(10)
# df.sort_values(by='pagerank', ascending=False).head(10)

**What URL has the second highest pagerank? Does it make sense or is that a bug of `surfer()`?**

## Why pagerank wins
It may not be clear that pagerank is better than degree sorting. After all, sites like umich.edu and eecs.umich.edu rise to the top in all three rankings. To see why pagerank is superior, consider the site http://eecs.umich.edu/cse:

In [ ]:
df.loc['http://eecs.umich.edu/cse']

In our graph, this site has an in degree of 1, an out degree of 69, and a pagerank value of 0.002386. It is near the bottom of the list on all three rankings.

Now suppose http://umich.edu and http://regents.umich.edu linked to http://eecs.umich.edu/cse. This would indicate that http://eecs.umich.edu/cse is a very important site. Would our rankings capture that?

In [ ]:
# add our hypothetical graph edges:
G.add_edge('http://umich.edu','http://eecs.umich.edu/cse')
G.add_edge('http://regents.umich.edu','http://eecs.umich.edu/cse')

# recompute adjacency matrix:
A2 = nx.adjacency_matrix(G, links).todense()

# compute transition matrix Ha:
H2 = normalize_rows(fix_dangling_nodes(A2))
alpha = 0.85
Ha2 = remove_zeros(H2, alpha).T

# perform power iteration:
r2 = power_iteration(Ha2, 200)

# update data table:
din_vec = [G.in_degree()[l] for l in links]
dout_vec = [G.out_degree()[l] for l in links]

df2 = pd.DataFrame({ 'in_degree' : din_vec,
                    'out_degree' : dout_vec,
                    'pagerank' : r2 }, index=links)

df2.sort_values(by='pagerank', ascending=False).head(10)

In [ ]:
df2.loc['http://eecs.umich.edu/cse']

So what happened?

* In degree increased from 1 to 3, a modest increase.
* Out degree remained the same.
* Pagerank value increased from 0.002386 to 0.19197, a significant boost!

Despite being linked to by two of the most important pages, http://eecs.umich.edu/cse did not reach the top of in degree or out degree rankings. But its pagerank boost was huge: its new value of 0.191969 makes it the 5th highest site by pagerank!